<a href="https://colab.research.google.com/github/tarvt/DeleSmell_SA_G5/blob/main/dl_compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras_self_attention

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18913 sha256=0fbf7701cdf1c40f58e2afc5cf88f17c7f110fdddcc9c6895e1b32e9cad9c6c5
  Stored in directory: /root/.cache/pip/wheels/ac/13/2d/3de7c76f618a8d162884ac5b726a8c2242ad88afa370f1e62f
Successfully built keras-self-attention


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import regularizers
from keras.layers import Dense,Flatten,Conv1D,MaxPooling1D,Dropout,BatchNormalization
from tensorflow.keras import Sequential
from tensorflow.keras import Model
from keras import regularizers
from keras import initializers
from keras_self_attention import SeqSelfAttention, SeqWeightedAttention
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from imblearn.over_sampling import SMOTE

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
dataset_path = '/content/drive/MyDrive/delesmell/METHOD_01.csv'
df = pd.read_csv(dataset_path)
df.head()

,NAME,ALD,ATFD,CALIN,CC,CCL,CDISP,CEXT,CINT,CM,...,LOC,MAXNESTING,NOAV,NOEU,NOLV,NOP,word2vec,word2vec2,word2vec3,is_brain
0,tagDefineTextField,1,0,0,0,2,0.00,0,0,0,...,5,1,2,0,0,14,0.766091,0.295311,-0.482078,False
1,tagDefineFont2,1,0,1,0,2,0.00,0,0,0,...,6,1,3,0,0,13,0.350905,0.692511,0.782078,False
2,classifyStream,0,0,32,0,3,0.00,0,0,0,...,118,3,34,0,27,11,-0.064280,1.089711,2.046234,False
3,classifyStream,0,0,17,0,3,0.17,1,6,0,...,76,3,16,0,13,10,-0.479466,1.486911,3.310390,False
4,generate,0,0,22,0,3,0.00,0,0,0,...,131,3,37,0,20,9,-0.894651,1.884111,4.574546,False


In [10]:
x, y = df.iloc[:, 1:-1], df.iloc[:, -1]
x.head()

,ALD,ATFD,CALIN,CC,CCL,CDISP,CEXT,CINT,CM,CYCLO,...,LAA,LOC,MAXNESTING,NOAV,NOEU,NOLV,NOP,word2vec,word2vec2,word2vec3
0,1,0,0,0,2,0.00,0,0,0,2,...,1.0,5,1,2,0,0,14,0.766091,0.295311,-0.482078
1,1,0,1,0,2,0.00,0,0,0,2,...,1.0,6,1,3,0,0,13,0.350905,0.692511,0.782078
2,0,0,32,0,3,0.00,0,0,0,16,...,1.0,118,3,34,0,27,11,-0.064280,1.089711,2.046234
3,0,0,17,0,3,0.17,1,6,0,17,...,1.0,76,3,16,0,13,10,-0.479466,1.486911,3.310390
4,0,0,22,0,3,0.00,0,0,0,9,...,1.0,131,3,37,0,20,9,-0.894651,1.884111,4.574546


In [11]:
len(y)

90315

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [13]:
x_train = x_train.values.reshape((x_train.shape[0],x_train.shape[1],1))
x_test = x_test.values.reshape((x_test.shape[0], x_test.shape[1],1))

In [14]:
onehot = OneHotEncoder(sparse=False)
y_train = onehot.fit_transform(y_train.values.reshape(len(y_train), 1))
y_test = onehot.fit_transform(y_test.values.reshape(len(y_test), 1))

print(x_train.shape)
print(y_train.shape)

(72252, 24, 1)
(72252, 2)


In [15]:
x_train.shape[1:]

(24, 1)

In [16]:
type(x_train)


numpy.ndarray

In [17]:
model = Sequential()

model.add(Dense(64,activation='relu',input_shape=x_train.shape[1:]))
model.add(Dropout(0.5))
model.add(Dense(128,activation='tanh'))
model.add(Flatten())
model.add(Dense(2,activation='softmax'))

In [18]:
inp=Input(shape=(x_train.shape[1:]))

x = Conv1D(32, 6, padding = "same", activation='tanh')(inp)
x = BatchNormalization()(x)
x = Conv1D(64, kernel_size=6, strides=1, padding='same',activation='relu')(x)
x = Conv1D(64, 6, strides=1, padding='same',activation='relu')(x)
x = LeakyReLU(alpha=0.33)(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Conv1D(128, kernel_size=6, strides=1, padding='same',activation='relu')(x)
x = Conv1D(128, 6, strides=1, padding='same',activation='relu')(x)
x = LeakyReLU(alpha=0.33)(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Conv1D(256, kernel_size=6, strides=1, padding='same',activation='relu')(x)
x = Conv1D(256, 6, strides=1, padding='same',activation='relu')(x)
x = LeakyReLU(alpha=0.33)(x)
x = Dropout(0.5)(x)
x = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.001))(x)
g = Flatten()(x)
print("g=",x)




g= KerasTensor(type_spec=TensorSpec(shape=(None, 24, 512), dtype=tf.float32, name=None), name='dense_3/Relu:0', description="created by layer 'dense_3'")


In [19]:
#Specify parameters such as data dimension and time step
data_dim = 1
timesteps = 4

lstm1=LSTM(units=32,activation='tanh',return_sequences=True)(inp)
l=Dropout(0.5)(lstm1)
l=Dense(64,activation="relu")(l)
l=Dropout(0.3)(l)
l=Dense(128,activation="relu")(l)
l=Dropout(0.3)(l)
d = Flatten()(l)
print("d=",l)

g2=concatenate([g,d],axis=1)
q=Dense(128)(g2)
q=LeakyReLU(alpha=0.33)(q)
q=Dropout(0.5)(q)
q=LeakyReLU(alpha=0.33)(q)
q=Dropout(0.5)(q)
q=LeakyReLU(alpha=0.33)(q)
q=Dropout(0.5)(q)
q=Dense(256,activation="tanh")(q)

output = Dense(2, kernel_regularizer=tf.keras.regularizers.l2(0.01),activation='softmax')(q)

model = Model(inputs=inp,outputs=output)

d= KerasTensor(type_spec=TensorSpec(shape=(None, 24, 128), dtype=tf.float32, name=None), name='dropout_6/Identity:0', description="created by layer 'dropout_6'")


In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 24, 1)]      0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 24, 32)       224         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 24, 32)      128         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 conv1d_1 (Conv1D)              (None, 24, 64)       12352       ['batch_normalization[0][0]']

In [21]:
model.compile(optimizer = tf.keras.optimizers.Adam(0.001),  #optimizer
#               loss ='squared_hinge',
              loss = 'categorical_crossentropy', #loss function
              metrics = ['accuracy']
             )

In [22]:
#Monitor the learning rate, when the val_loss does not drop 3 times, the learning rate is multiplied by 0.5, and the minimum does not exceed min_lr
lr_reduce=keras.callbacks.ReduceLROnPlateau('val_loss',patience=3,factor=0.5,min_lr=0.000001)

In [23]:
history = model.fit(x_train,
                    y_train,
                    epochs=60,
                    callbacks = [lr_reduce],
                    batch_size= 16,
                    validation_data = (x_test,y_test),
                    shuffle=True)

Epoch 1/60
4516/4516 [==============================] - 494s 109ms/step - loss: nan - accuracy: 0.8356 - val_loss: nan - val_accuracy: 0.8445 - lr: 0.0010
Epoch 2/60
4516/4516 [==============================] - 471s 104ms/step - loss: nan - accuracy: 0.8418 - val_loss: nan - val_accuracy: 0.8445 - lr: 0.0010
Epoch 3/60
4516/4516 [==============================] - 478s 106ms/step - loss: nan - accuracy: 0.8418 - val_loss: nan - val_accuracy: 0.8445 - lr: 0.0010
Epoch 4/60
4516/4516 [==============================] - 479s 106ms/step - loss: nan - accuracy: 0.8418 - val_loss: nan - val_accuracy: 0.8445 - lr: 5.0000e-04
Epoch 5/60
4516/4516 [==============================] - 477s 106ms/step - loss: nan - accuracy: 0.8418 - val_loss: nan - val_accuracy: 0.8445 - lr: 5.0000e-04
Epoch 6/60
4516/4516 [==============================] - 470s 104ms/step - loss: nan - accuracy: 0.8418 - val_loss: nan - val_accuracy: 0.8445 - lr: 5.0000e-04
Epoch 7/60
3980/4516 [=========================>....] - ET

KeyboardInterrupt: ignored

In [ ]:
score = model.evaluate(x_train, y_train, verbose=0)
print('train loss:', score[0])
print('train accuracy:', score[1])